# Text Summarization

In [3]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [0]:
%%bash
pip install --upgrade pandas --force-reinstall
pip install -U spacy==2.2.4 requests==2.23.0 folium==0.2.1
pip install -U nltk==3.4.5 rouge==0.3.1
pip install --upgrade razdel fasttext networkx
pip install --upgrade torch transformers catalyst pymorphy2 pymorphy2-dicts-ru
pip install --upgrade deeppavlov

In [0]:
import random
import copy
from itertools import combinations

from tqdm.notebook import tqdm

import pandas as pd
import math
import numpy as np

from scipy.spatial import distance
import networkx as nx
import fasttext
import razdel
import nltk, pymorphy2

import torch, catalyst, transformers
from transformers import BertTokenizer, BertForSequenceClassification

from torch.utils import data
import torch.nn as nn
import torch.nn.functional as F
from torch.optim.lr_scheduler import CyclicLR

import catalyst
from catalyst.dl.runner import SupervisedRunner
from catalyst.core.callbacks.early_stop import EarlyStoppingCallback

from nltk.translate.bleu_score import corpus_bleu
from rouge import Rouge

In [0]:
from deeppavlov.core.common.file import read_json
from deeppavlov import build_model, configs
from deeppavlov.models.embedders.elmo_embedder import ELMoEmbedder

In [6]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
DEVICE

device(type='cuda')

In [0]:
%%bash
wget -q https://www.dropbox.com/s/43l702z5a5i2w8j/gazeta_train.txt
wget -q https://www.dropbox.com/s/k2egt3sug0hb185/gazeta_val.txt
wget -q https://www.dropbox.com/s/3gki5n5djs9w0v6/gazeta_test.txt

In [0]:
def read_gazeta_records(file_name, shuffle=True, sort_by_date=False):
    assert shuffle != sort_by_date
    records = []
    with open(file_name, "r") as r:
        for line in r:
            records.append(eval(line)) # Simple hack
    records = pd.DataFrame(records)
    if sort_by_date:
        records = records.sort("date")
    if shuffle:
        records = records.sample(frac=1)
    return records

In [0]:
train_records = read_gazeta_records("gazeta_train.txt")
val_records = read_gazeta_records("gazeta_val.txt")
test_records = read_gazeta_records("gazeta_test.txt")

In [0]:
(train_records.shape, test_records.shape, val_records.shape)

((52400, 5), (5770, 5), (5265, 5))

In [0]:
# check empty records
def check_empty(records):
    for idx, row in records.iterrows():
        try:
            if not (len(row['text']) and len(row['summary'])):
                print(row['text'], row['summary'])
            except AttributeError:
                  print(f'Empty object found at {idx}!')

In [0]:
check_empty(train_records)
check_empty(val_records)
check_empty(test_records)

In [0]:
# get average text len (# sentences), sentence len (# words)
def avg_text_sum_len(records):
    text_len = 0; sum_len = 0
    text_words = 0; sum_words = 0
    for text, summary in tqdm(records[['text', 'summary']].values):
        text_sent = [sent.text for sent in razdel.sentenize(text)]
        sum_sent = [sent.text for sent in razdel.sentenize(summary)]
        text_len += len(text_sent)
        sum_len += len(sum_sent)
        text_words += np.max([len(list(razdel.tokenize(sent))) for sent in text_sent])
        sum_words += np.max([len(list(razdel.tokenize(sent))) for sent in sum_sent])
    n = records.shape[0]
    text_len /= n; sum_len /= n
    text_words /= n; sum_words /= n
    return text_len, text_words, sum_len, sum_words

In [0]:
avg_text_sum_len(train_records)

(37.24198473282443, 50.627614503816794, 2.656583969465649, 24.981068702290077)

In [0]:
def calc_scores(references, predictions, metric="all"):
    print("Count:", len(predictions))
    print("Ref:", references[-1])
    print("Hyp:", predictions[-1])

    if metric in ("bleu", "all"):
        print("BLEU: ", corpus_bleu([[r] for r in references], predictions))
    if metric in ("rouge", "all"):
        rouge = Rouge()
        scores = rouge.get_scores(predictions, references, avg=True)
        print("ROUGE: ", scores)

## TextRank

In [0]:
def gen_text_rank_summary(text, calc_sim, summary_part=0.1, lower=True, 
                          tokenize=True, morph=None, model=None, get_vecs_f=None):
    # split text
    sentences = [sentence.text.lower() if lower else sentence.text \
                 for sentence in razdel.sentenize(text)]
    n_sentences = len(sentences)

    # tokenization
    if tokenize:
        sent_words = [[token.text.lower() if lower else token.text \
                       for token in razdel.tokenize(sentence)] for sentence in sentences]
    else:
        sent_words = sentences

    # lemmatization
    if morph is not None:
        sent_words = [[morph.parse(word)[0].normal_form for word in words] \
                           for words in sent_words]

    # pairwise sentence similarity
    pairs = combinations(range(n_sentences), 2)
    if model is not None and get_vecs_f is not None:
        sent_words = get_vecs_f(model, sent_words)
    scores = [(i, j, calc_sim(sent_words[i], sent_words[j])) for i, j in pairs]

    # weighted graph with similarity scores
    g = nx.Graph()
    g.add_weighted_edges_from(scores)

    # PageRank
    pr = nx.pagerank(g)
    result = [(i, pr[i], s) for i, s in enumerate(sentences) if i in pr]
    result.sort(key=lambda x: x[1], reverse=True)

    # choose top sent
    n_summary_sentences = max(int(n_sentences * summary_part), 1)
    result = result[:n_summary_sentences]

    # restore original sent order
    result.sort(key=lambda x: x[0])

    # restore summary text
    predicted_summary = " ".join([sentence for i, proba, sentence in result])
    predicted_summary = predicted_summary.lower() if lower else predicted_summary
    return predicted_summary

In [0]:
def calc_text_rank_score(records, calc_sim, summary_part=0.1, lower=True, tokenize=True, 
                         nrows=1000, morph=None, model=None, get_vecs_f=None):
    references = []
    predictions = []

    for text, summary in records[['text', 'summary']].values[:nrows]:
        summary = summary if not lower else summary.lower()
        references.append(summary)

        predicted_summary = gen_text_rank_summary(text, calc_sim, summary_part, 
                                                  lower, tokenize, morph=morph, 
                                                  model=model, 
                                                  get_vecs_f=get_vecs_f)
        text = text if not lower else text.lower()
        predictions.append(predicted_summary)

    calc_scores(references, predictions)

### Similarity based on words intersection

In [0]:
def unique_words_sim(words1, words2):
    words1 = set(words1)
    words2 = set(words2)
    if not len(words1) or not len(words2):
        return 0.0
    return len(words1.intersection(words2))/(np.log10(len(words1)) + np.log10(len(words2)))

In [0]:
calc_text_rank_score(test_records, calc_sim=unique_words_sim)

Count: 1000
Ref: в крыму заявили, что угрозы украинских властей отреагировать на запуск поездов по крымскому мосту являются попыткой вмешательства во внутренние дела россии. при этом власти полуострова напомнили, что такие обещания киева пустые, и, как правило, не влекут никаких последствий.
Hyp: по его словам, подобные обещания, как были высказаны так называемым «представителем президента украины в крыму» антоном кориневичем, являются голословными и вряд ли представляют реальную угрозу для моста через керченский пролив, передает rt. доехать на именных поездах «таврия» из москвы до симферополя в купе можно будет за 2966 руб., а из санкт-петербурга до севастополя — за 3906 руб. «пять одноэтажных поездов, состоящих из купейных и плацкартных вагонов выведут на маршрут, который соединит северную столицу россии с севастополем», — говорится в сообщении перевозчика.
BLEU:  0.27401385077499585
ROUGE:  {'rouge-1': {'f': 0.15987203393568736, 'p': 0.13245879938563265, 'r': 0.21737094280809163}, '


---

### Cosine similarity with pretrained embeddings

In [0]:
def cosine_sim(s1, s2):
    return 1 - distance.cosine(s1, s2)

#### FastText Embeddings

In [0]:
def get_ft_embeddings(model, sent_words):
    sent_vecs = []
    for sent in sent_words:
        word_vecs = [model.get_word_vector(word) for word in sent]
        sent_vecs.append(np.mean(word_vecs, axis=0))
    return sent_vecs

In [0]:
ft = fasttext.load_model('drive/My Drive/ft_lem.bin')

In [0]:
morph = pymorphy2.MorphAnalyzer()

In [0]:
calc_text_rank_score(test_records, calc_sim=cosine_sim, model=ft, 
                     get_vecs_f=get_ft_embeddings, morph=morph)

Count: 1000
Ref: в крыму заявили, что угрозы украинских властей отреагировать на запуск поездов по крымскому мосту являются попыткой вмешательства во внутренние дела россии. при этом власти полуострова напомнили, что такие обещания киева пустые, и, как правило, не влекут никаких последствий.
Hyp: уже в следующем году, когда появится стабильное расписание, люди из всех уголков нашей страны смогут заранее планировать поездки в крым. севастопольцы очень ждут запуска поездов, чтобы заранее планировать свои поездки в столицу и другие регионы», — заверил глава города федерального значения. стоит отметить, что чуть ранее глава республики крым сергей аксенов заявил: инфраструктура полуострова полностью готова к возобновлению железнодорожного сообщения с украиной, теперь решение остается только за киевом.
BLEU:  0.2676917384164895
ROUGE:  {'rouge-1': {'f': 0.15266251535370307, 'p': 0.12352676093801424, 'r': 0.2153909542432146}, 'rouge-2': {'f': 0.03135829505675102, 'p': 0.02519721936393644, 'r'

#### ELMo Embeddings

In [0]:
def get_elmo_embeddings(model, sent_words):
    return model(sent_words)

In [0]:
elmo = ELMoEmbedder('http://files.deeppavlov.ai/deeppavlov_data/elmo_ru-news_wmt11-16_1.5M_steps.tar.gz')

In [0]:
calc_text_rank_score(test_records, calc_sim=cosine_sim, model=elmo, 
                     get_vecs_f=get_elmo_embeddings, tokenize=True)

Count: 1000
Ref: в крыму заявили, что угрозы украинских властей отреагировать на запуск поездов по крымскому мосту являются попыткой вмешательства во внутренние дела россии. при этом власти полуострова напомнили, что такие обещания киева пустые, и, как правило, не влекут никаких последствий.
Hyp: кроме того, сенатор предположил, что возможность добраться до полуострова на поезде значительно увеличит количество отдыхающих в крыму. севастопольцы очень ждут запуска поездов, чтобы заранее планировать свои поездки в столицу и другие регионы», — заверил глава города федерального значения. стоит отметить, что чуть ранее глава республики крым сергей аксенов заявил: инфраструктура полуострова полностью готова к возобновлению железнодорожного сообщения с украиной, теперь решение остается только за киевом.
BLEU:  0.2936942520323366
ROUGE:  {'rouge-1': {'f': 0.15978593060486648, 'p': 0.1366146967958347, 'r': 0.20728347462094615}, 'rouge-2': {'f': 0.03369622113710133, 'p': 0.028526382895340662, 'r'

#### RuBERT Embeddings

In [0]:
%%bash
wget -O rubert_sent.tar.gz http://files.deeppavlov.ai/deeppavlov_data/bert/sentence_ru_cased_L-12_H-768_A-12_pt.tar.gz
mkdir -p rubert_sent && tar -C rubert_sent/ -zxvf rubert_sent.tar.gz --strip-components=1

In [0]:
def get_rubert_vectors(model, texts):
    tokens, token_embs, subtokens, subtoken_embs, sent_max_embs, \
    sent_mean_embs, bert_pooler_outputs = model(texts)
    return sent_mean_embs

In [0]:
bert_config = read_json(configs.embedder.bert_embedder)

In [0]:
bert_config['metadata']['variables']['BERT_PATH'] = 'rubert_sent/'

In [0]:
rubert_sent = build_model(bert_config)

In [0]:
calc_text_rank_score(test_records, calc_sim=cosine_sim, model=rubert_sent, 
                     get_vecs_f=get_rubert_vectors, tokenize=False)

Count: 1000
Ref: в крыму заявили, что угрозы украинских властей отреагировать на запуск поездов по крымскому мосту являются попыткой вмешательства во внутренние дела россии. при этом власти полуострова напомнили, что такие обещания киева пустые, и, как правило, не влекут никаких последствий.
Hyp: член комитета совета федерации по международным делам, сенатор от крыма сергей цеков указал, что такой резкий старт свидетельствует об интересе пассажиров к поездкам в крым на железнодорожном транспорте. стоит отметить, что чуть ранее глава республики крым сергей аксенов заявил: инфраструктура полуострова полностью готова к возобновлению железнодорожного сообщения с украиной, теперь решение остается только за киевом. по словам чиновника, республика готова запустить поезда при согласии президента россии владимира путина .
BLEU:  0.30728739890818507
ROUGE:  {'rouge-1': {'f': 0.15850907546303156, 'p': 0.13916826986068476, 'r': 0.19688784244868618}, 'rouge-2': {'f': 0.03620491586594866, 'p': 0.031

## Extractive RNN


Implemented approach (except that the sentence encoder GRU were replaced by CNN): https://arxiv.org/pdf/1611.04230.pdf

### Extractive training

In [0]:
def build_oracle_summary_greedy(text, gold_summary, calc_score, lower=True, max_sentences=30):
    gold_summary = gold_summary.lower() if lower else gold_summary
    # split text
    sentences = [sentence.text.lower() if lower else sentence.text \
                 for sentence in razdel.sentenize(text)][:max_sentences]
    n_sentences = len(sentences)
    oracle_summary_sentences = set()
    score = -1.0
    summaries = []
    for _ in range(min(n_sentences, 20)):
        for i in range(n_sentences):
            if i in oracle_summary_sentences:
                continue
            current_summary_sentences = copy.copy(oracle_summary_sentences)
            # add sentences to the generated summary
            current_summary_sentences.add(i)
            current_summary = " ".join([sentences[index] \
                                        for index in sorted(list(current_summary_sentences))])
            # compute score
            current_score = calc_score(current_summary, gold_summary)
            summaries.append((current_score, current_summary_sentences))
        # if the added sentence improved score, add new sentences (break otherwise)
        best_summary_score, best_summary_sentences = max(summaries)
        if best_summary_score <= score:
            break
        oracle_summary_sentences = best_summary_sentences
        score = best_summary_score
    oracle_sorted = sorted(list(oracle_summary_sentences))
    oracle_summary = " ".join([sentences[index] for index in oracle_sorted])
    return oracle_summary, oracle_summary_sentences

def calc_single_score(pred_summary, gold_summary, rouge):
    return rouge.get_scores([pred_summary], [gold_summary], avg=True)['rouge-2']['f']

In [0]:
def calc_oracle_score(records, nrows=1000, lower=True):
    references = []
    predictions = []
    rouge = Rouge()
    calc_f = lambda x, y: calc_single_score(x, y, rouge)
  
    for text, summary in tqdm(records[['text', 'summary']].values[:nrows]):
        summary = summary if not lower else summary.lower()
        references.append(summary)
        predicted_summary, _ = build_oracle_summary_greedy(text, summary, 
                                                           calc_score=calc_f)
        predictions.append(predicted_summary)

    calc_scores(references, predictions)

In [0]:
calc_oracle_score(test_records, nrows=test_records.shape[0])


Count: 5770
Ref: обновление ios под номером 13.1.2, призванное решить ряд проблем в системе, лишь прибавило головной боли владельцам iphone — с гаджетов массово сбрасываются звонки, а батарея очень быстро теряет заряд. эксперты рекомендуют не скачивать апдейт, а дождаться, пока apple исправит все недочеты.
Hyp: владельцы «яблочных» гаджетов массово жалуются на ряд проблем, которые возникли после того, как на их смартфоны был установлен апдейт ios 13.1.2, сообщает forbes.
BLEU:  0.5378135116343372
ROUGE:  {'rouge-1': {'f': 0.3718077014808164, 'p': 0.40694068294010777, 'r': 0.36938828858749356}, 'rouge-2': {'f': 0.21086750192505038, 'p': 0.23672926902092492, 'r': 0.20795293543045332}, 'rouge-l': {'f': 0.3254315265705807, 'p': 0.3783154260781943, 'r': 0.34271136213185266}}


In [0]:
def add_oracle_summary_to_records(records, max_sentences=40, lower=True, nrows=1000):
    rouge = Rouge()
    sentences_ = []
    oracle_sentences_ = []
    oracle_summary_ = []
    records = records.iloc[:nrows].copy()
    calc_f = lambda x, y: calc_single_score(x, y, rouge)

    for text, summary in tqdm(records[['text', 'summary']].values):
        summary = summary.lower() if lower else summary
        sentences = [sentence.text.lower() if lower else sentence.text \
                     for sentence in razdel.sentenize(text)][:max_sentences]
        oracle_summary, sentences_indicies =\
                    build_oracle_summary_greedy(text, summary, calc_score=calc_f,
                    lower=lower, max_sentences=max_sentences)
        sentences_ += [sentences]
        oracle_sentences_ += [list(sentences_indicies)]
        oracle_summary_ += [oracle_summary]
    records['sentences'] = sentences_
    records['oracle_sentences'] = oracle_sentences_
    records['oracle_summary'] = oracle_summary_
    return records

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
ext_train_records = add_oracle_summary_to_records(train_records, nrows=train_records.shape[0])
ext_train_records.to_pickle('drive/My Drive/train.pkl')

In [0]:
ext_val_records = add_oracle_summary_to_records(val_records, nrows=val_records.shape[0])
ext_val_records.to_pickle('drive/My Drive/val.pkl')

In [0]:
ext_test_records = add_oracle_summary_to_records(test_records, nrows=test_records.shape[0])
ext_test_records.to_pickle('drive/My Drive/test.pkl')

In [0]:
ext_train_records.iloc[0]

url                 https://www.gazeta.ru/sport/2011/10/a_3807194....
text                После прихода в «Спартак» Марцела Госсы в кома...
title                  «Спартаковские болельщики всегда будут роднее»
summary             Нападающий «Атланта» Бранко Радивоевич причино...
date                                              2011-10-20 01:52:50
sentences           [после прихода в «спартак» марцела госсы в ком...
oracle_sentences                                              [2, 34]
oracle_summary      лишним оказался капитан команды бранко радивое...
Name: 28004, dtype: object

In [0]:
ext_train_records = pd.read_pickle('drive/My Drive/train.pkl')
ext_val_records = pd.read_pickle('drive/My Drive/val.pkl')
ext_test_records = pd.read_pickle('drive/My Drive/test.pkl')

### Pretrained BERT model and tokenizer

In [0]:
url = "http://files.deeppavlov.ai/deeppavlov_data/bert/rubert_cased_L-12_H-768_A-12_pt.tar.gz"

In [0]:
%%bash -s "$url"
wget -O rubert.tar.gz $1
mkdir -p rubert && tar -C rubert/ -zxvf rubert.tar.gz --strip-components=1
mv rubert/bert_config.json rubert/config.json
cp rubert.tar.gz "drive/My Drive/rubert.tar.gz"

In [0]:
%%bash
mkdir -p rubert && tar -C rubert/ -zxvf "drive/My Drive/rubert.tar.gz" --strip-components=1
mv rubert/bert_config.json rubert/config.json

In [0]:
tokenizer = BertTokenizer.from_pretrained('rubert')
bert = BertForSequenceClassification.from_pretrained('rubert')

### Dataloaders

In [0]:
class ExtDataset(data.Dataset):
    def __init__(self, records, tokenizer, lower=True, max_sent=40, 
                 max_sent_len=50, device=torch.device('cpu')):
        self.records = records
        self.num_samples = records.shape[0]
        self.tokenizer = tokenizer
        self.lower = lower
        self.rouge = Rouge()
        self.max_sent = max_sent
        self.max_sent_len = max_sent_len
        self.device = device

    def __len__(self):
        return self.num_samples

    def __getitem__(self, idx):
        cur_rec = self.records.iloc[idx]
        inputs = [tokenizer.encode(sent, max_length=self.max_sent_len) \
                  for sent in cur_rec.sentences]
        outputs = [int(i in cur_rec.oracle_sentences) \
                   for i in range(len(cur_rec.sentences))]
        return {'inputs': inputs, 'outputs': outputs}

In [0]:
def collate_fn(records):
    max_length = max(len(sentence) for record in records for sentence in record['inputs'])
    max_sentences = max(len(record['outputs']) for record in records)

    new_inputs = torch.zeros((len(records), max_sentences, max_length))
    new_outputs = torch.zeros((len(records), max_sentences))
    for i, record in enumerate(records):
        for j, sentence in enumerate(record['inputs']):
            new_inputs[i, j, :len(sentence)] += np.array(sentence)
        new_outputs[i, :len(record['outputs'])] += np.array(record['outputs'])
    return {'features': new_inputs.type(torch.LongTensor), 'targets': new_outputs}

In [0]:
BATCH_SIZE = 32
TRAIN_SIZE = 5000
VALID_SIZE = 500

train_loaders = {
    'train': data.DataLoader(ExtDataset(ext_train_records.iloc[:TRAIN_SIZE, :], tokenizer), 
                             batch_size=BATCH_SIZE, collate_fn=collate_fn, 
                             shuffle=True),
    'valid': data.DataLoader(ExtDataset(ext_val_records.iloc[:VALID_SIZE, :], tokenizer), 
                             batch_size=BATCH_SIZE, collate_fn=collate_fn)}
test_loader = data.DataLoader(ExtDataset(ext_test_records.iloc[:VALID_SIZE, :], tokenizer), 
                              batch_size=1, collate_fn=collate_fn)


### SummaRuNNer

#### Sentence encoder

In [0]:
def cnn_block(in_channels, out_channels, kernel, n=2):
    layers = []
    for _ in range(n):
        conv = nn.Conv1d(in_channels, out_channels, kernel)
        bn = nn.BatchNorm1d(out_channels)
        layers += [conv, bn, nn.LeakyReLU(inplace=True)]
        in_channels = out_channels
    return nn.Sequential(*layers)

In [0]:
class SentenceEncoderCNN(nn.Module):
    def __init__(self, emb_layer, emb_dim, out_c, kernels):
        super(SentenceEncoderCNN, self).__init__()
        self.emb_dim = emb_dim
        self.out_c = out_c
        self.embedding = emb_layer
        self.convs = nn.ModuleList([cnn_block(self.emb_dim, self.out_c, kernel) \
                                  for kernel in kernels])
    def forward(self, inputs):
        # [B x Len x Emb] to [B x Emb x Len] (for conv by len)
        emb = F.dropout(self.embedding(inputs), p=0.2).permute((0, 2, 1))
        n_gram_emb = []
        for conv in self.convs:
            new_emb = conv(emb) # [B x out_c x (Len - out_c) / stride + 1]
            # [B x out_c x 1]
            new_emb = F.max_pool1d(new_emb, new_emb.size(2)).squeeze(2)
            n_gram_emb += [new_emb]
        n_gram_emb = torch.cat(n_gram_emb, 1).view(-1, self.emb_dim, len(self.convs))
        return F.avg_pool1d(n_gram_emb, n_gram_emb.size(2)).squeeze(2)

#### Sentence tagger

In [0]:
class SentenceTaggerRNN(nn.Module):
    def __init__(self, emb_layer, kernels, hid_dim=768, bidirect=True, 
                 max_sent_len=50, n_segments=10, pos_emb_dim=5):
        super(SentenceTaggerRNN, self).__init__()
        
        tok_emb_dim = emb_layer.weight.shape[1]
        # for cat without pooling tok_emb_dim - tok_emb_dim % len(kernels)
        sent_enc_hid_dim = tok_emb_dim

        self.hid_dim = hid_dim
        self.n_dir = bidirect + 1
        self.max_sent_len = max_sent_len
        self.n_segments = n_segments

        # for cat without pooling sent_enc_hid_dim // len(kernels)
        self.sent_enc = SentenceEncoderCNN(emb_layer, tok_emb_dim, 
                                           tok_emb_dim, kernels)                                 
        self.rnn = nn.GRU(sent_enc_hid_dim, hid_dim, bidirectional=bidirect, 
                          batch_first=True)
        
        self.doc_layer = nn.Linear(self.hid_dim, self.hid_dim)
        self.content = nn.Linear(self.hid_dim, 1, bias=False)
        self.salience = nn.Bilinear(self.hid_dim, self.hid_dim, 1, bias=False)
        self.novelty = nn.Bilinear(self.hid_dim, self.hid_dim, 1, bias=False)

        self.abs_pos_emb = nn.Embedding(max_sent_len, pos_emb_dim)
        self.rel_pos_emb = nn.Embedding(n_segments, pos_emb_dim)

        self.abs_pos = nn.Linear(pos_emb_dim, 1, bias=False)
        self.rel_pos = nn.Linear(pos_emb_dim, 1, bias=False)
        self.bias = nn.Parameter(torch.FloatTensor(1).uniform_(-0.1, 0.1))

    def forward(self, inputs):
        batch_size, n_sent, n_tokens = inputs.size()
        # to [Batch * Doc_len x Sent_len]
        inputs = inputs.view(-1, n_tokens)
        # [Batch * Doc_len x Hid_dim] to [B x Doc_Len x Hid_dim]
        sent_emb = self.sent_enc(inputs).view(batch_size, n_sent, -1)

        # [B x Doc_len x Hid_dim * n_directions] to [B x Doc_len x n_directions x Hid_dim]
        sent_emb = self.rnn(sent_emb)[0].view(batch_size, n_sent, self.n_dir, -1)
        sent_emb = F.dropout(sent_emb, p=0.2)

        # mean by directions, [B x Doc_len x Hid_dim]
        avg_sent_emb = torch.mean(sent_emb, dim=2)
        # mean by document len, expand for compability with @avg_sent_emb
        doc_emb = torch.mean(avg_sent_emb, dim=1, keepdim=True).expand(batch_size, n_sent, -1)

        # total document representation, [B x Doc_len x Hid_dim]
        D = torch.tanh(self.doc_layer(doc_emb))

        # initial summary state
        h0 = torch.zeros((batch_size, 1, self.hid_dim), device=DEVICE)
        # partial summary representations (except very last sentence)
        S = torch.cumsum(avg_sent_emb[:, :-1], dim=1)
        S = torch.cat((h0, S), dim=1)

        content = self.content(avg_sent_emb).squeeze(2)
        salience = self.salience(avg_sent_emb, D).squeeze(2)
        novelty = -1 * self.novelty(avg_sent_emb, torch.tanh(S)).squeeze(2)

        pos_idx = torch.arange(n_sent, dtype=torch.long, device=DEVICE).expand(batch_size, n_sent)
        abs_emb = self.abs_pos_emb(pos_idx)
        rel_emb = self.rel_pos_emb(pos_idx // self.n_segments)

        abs_p = self.abs_pos(abs_emb).squeeze(2)
        rel_p = self.rel_pos(abs_emb).squeeze(2)

        return content + salience + novelty + abs_p + rel_p

In [0]:
kernels = [2, 3, 5, 7]
tagger = SentenceTaggerRNN(bert.get_input_embeddings(), kernels)

### Training

In [0]:
lr = 1e-4
num_epochs = 10

optimizer = torch.optim.Adam(tagger.parameters(), lr=lr)
criterion = nn.BCEWithLogitsLoss()
runner = SupervisedRunner(device=DEVICE)

In [19]:
runner.train(
    model=tagger,
    optimizer=optimizer,
    loaders=train_loaders,
    logdir='./logs',
    num_epochs=num_epochs,
    criterion=criterion,
    verbose=True,
    load_best_on_end=True,
    callbacks=[EarlyStoppingCallback(patience=3)]
)

1/10 * Epoch (train): 100% 157/157 [04:32<00:00,  1.74s/it, loss=0.233]
1/10 * Epoch (valid): 100% 16/16 [00:17<00:00,  1.08s/it, loss=0.269]
[2020-05-27 02:46:09,553] 
1/10 * Epoch 1 (_base): lr=0.0001 | momentum=0.9000
1/10 * Epoch 1 (train): loss=0.7252
1/10 * Epoch 1 (valid): loss=0.2987
2/10 * Epoch (train): 100% 157/157 [04:29<00:00,  1.72s/it, loss=0.249]
2/10 * Epoch (valid): 100% 16/16 [00:17<00:00,  1.08s/it, loss=0.229]
[2020-05-27 02:52:50,531] 
2/10 * Epoch 2 (_base): lr=0.0001 | momentum=0.9000
2/10 * Epoch 2 (train): loss=0.3877
2/10 * Epoch 2 (valid): loss=0.2485
3/10 * Epoch (train): 100% 157/157 [04:29<00:00,  1.72s/it, loss=0.117]
3/10 * Epoch (valid): 100% 16/16 [00:17<00:00,  1.08s/it, loss=0.259]
[2020-05-27 02:59:33,915] 
3/10 * Epoch 3 (_base): lr=0.0001 | momentum=0.9000
3/10 * Epoch 3 (train): loss=0.2245
3/10 * Epoch 3 (valid): loss=0.2773
4/10 * Epoch (train): 100% 157/157 [04:29<00:00,  1.72s/it, loss=0.226]
4/10 * Epoch (valid): 100% 16/16 [00:17<00:00,  1

In [20]:
references = []
predictions = []
threshold_prob = 0.2
tagger.eval()
with torch.no_grad():
    for i, item in tqdm(enumerate(test_loader), total=VALID_SIZE):
        probs = torch.sigmoid(tagger(item['features'].to(DEVICE)))[0]
        record = ext_test_records.iloc[i]
        predicted_summary = []
        sorted_p, sorted_i = torch.sort(probs, descending=True)
        for (prob, i) in zip(sorted_p, sorted_i):
            if prob < threshold_prob:
                break
            predicted_summary.append(record['sentences'][i])
        if not predicted_summary:
            predicted_summary.append(record['sentences'][sorted_i[0]])
        predicted_summary = ' '.join(predicted_summary)
        references.append(record['summary'].lower())
        predictions.append(predicted_summary)

    calc_scores(references, predictions)


Count: 500
Ref: ученые выяснили, откуда взялись ровные трещины на поверхности энцелада, спутника сатурна. спутник пострадал из-за собственных приливов, которые испытывает, вращаясь по орбите.
Hyp: американские ученые разгадали загадку одного из самых необычных тел солнечной системы — энцелада.
BLEU:  0.4207066727688847
ROUGE:  {'rouge-1': {'f': 0.237196947948618, 'p': 0.27462540643633476, 'r': 0.2343851245459358}, 'rouge-2': {'f': 0.09782887887790095, 'p': 0.11505677857292236, 'r': 0.09754047857158932}, 'rouge-l': {'f': 0.19459520396188398, 'p': 0.24654315261738127, 'r': 0.2099648816565591}}
